In [132]:
from os.path import join
import keras
from keras.applications import Xception, VGG16, VGG19, ResNet50, InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from keras.models import Model, load_model
from keras.utils.np_utils import to_categorical

import pandas as pd
import csv
import os
import numpy as np
import json
import pickle
from matplotlib import pyplot as plt
import sys
sys.path.append("../data_preparation/")

from batch_generator import BatchGenerator, BatchSequence

from sklearn.metrics import recall_score, precision_score, f1_score

In [277]:
# -*- coding: utf-8 -*-
import tensorflow as tf

## Model Initializations

### VGG16

In [253]:
model_vgg16 = load_model('Pretrained-networks/vgg16/vgg16_3000.h5')

In [280]:
for i, layer in enumerate(model_vgg16.layers):
    model_vgg16.layers[i].trainable = False
    model_vgg16.layers[i].name = '{}_{}'.format(layer.name, 'vgg16')

In [281]:
vgg16_out = model_vgg16.output

### VGG19

In [256]:
model_vgg19 = load_model('Pretrained-networks/vgg19/VGG19.h5')

In [282]:
for i, layer in enumerate(model_vgg19.layers):
    model_vgg19.layers[i].trainable = False
    model_vgg19.layers[i].name = '{}_{}'.format(layer.name, 'vgg19')

In [283]:
vgg19_out = model_vgg19.output

### Xception

In [259]:
model_xception = load_model('Pretrained-networks/Xception/xception.h5')

In [284]:
for i, layer in enumerate(model_xception.layers):
    model_xception.layers[i].trainable = False
    model_xception.layers[i].name = '{}_{}'.format(layer.name, 'xception')

In [285]:
xception_out = model_xception.output

### Inception

In [262]:
model_inception = load_model('Pretrained-networks/inception/inceptionV3.h5')

In [286]:
for i, layer in enumerate(model_inception.layers):
    model_inception.layers[i].trainable = False
    model_inception.layers[i].name = '{}_{}'.format(layer.name, 'inception')

In [287]:
inception_out = model_inception.output

### ResNet50

In [265]:
model_resnet50 = load_model('Pretrained-networks/ResNet50/ResNet50.h5')

In [288]:
for i, layer in enumerate(model_resnet50.layers):
    model_resnet50.layers[i].trainable = False
    model_resnet50.layers[i].name = '{}_{}'.format(layer.name, 'resnet50')

In [289]:
resnet50_out = model_resnet50.output

In [268]:
model_vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1_vgg16 (InputLayer)   (None, 290, 290, 3)       0         
_________________________________________________________________
block1_conv1_vgg16 (Conv2D)  (None, 290, 290, 64)      1792      
_________________________________________________________________
block1_conv2_vgg16 (Conv2D)  (None, 290, 290, 64)      36928     
_________________________________________________________________
block1_pool_vgg16 (MaxPoolin (None, 145, 145, 64)      0         
_________________________________________________________________
block2_conv1_vgg16 (Conv2D)  (None, 145, 145, 128)     73856     
_________________________________________________________________
block2_conv2_vgg16 (Conv2D)  (None, 145, 145, 128)     147584    
_________________________________________________________________
block2_pool_vgg16 (MaxPoolin (None, 72, 72, 128)       0         
__________

## Concatenation

In [290]:
# Checking if the instance is a symbolic tensor
keras.backend.is_keras_tensor(vgg16_out)

True

In [291]:
merge_0 = keras.layers.concatenate([vgg16_out, vgg19_out, xception_out])

In [292]:
output = Dense(228, activation='sigmoid', name='main_output')(merge_0)

In [293]:
model = Model(inputs=[model_vgg16.input, model_vgg19.input, model_xception.input], outputs=output)

In [294]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1_xception (InputLayer)   (None, 290, 290, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1_xception (Conv2D)  (None, 144, 144, 32) 864         input_1_xception[0][0]           
__________________________________________________________________________________________________
block1_conv1_bn_xception (Batch (None, 144, 144, 32) 128         block1_conv1_xception[0][0]      
__________________________________________________________________________________________________
block1_conv1_act_xception (Acti (None, 144, 144, 32) 0           block1_conv1_bn_xception[0][0]   
__________________________________________________________________________________________________
block1_con

# Testing

In [274]:
# Set the paths
input_path = os.path.abspath('../../mlipdata/')
images_path_train = os.path.join(input_path, 'files/train/')
images_path_validation = os.path.join(input_path, 'files/validation/')

In [295]:
# Load the multilabel binarizer
with open('pickles/binarizer.pickle', 'rb') as pickle_file:
    binarizer = pickle.load(pickle_file)

# Load training data from file
train={}
with open(os.path.join(input_path, 'train.json')) as json_data:
    train= json.load(json_data)

train_img_url = train['images']
train_img_url = pd.DataFrame(train_img_url)
train_ann = train['annotations']
train_ann = pd.DataFrame(train_ann)
train = pd.merge(train_img_url, train_ann, on='imageId', how='inner')
train['imageId'] = train['imageId'].astype(np.uint32)

y_train = np.array(train.labelId)
y_train_bin = binarizer.transform(y_train)

## Load validation
validation={}
with open(os.path.join(input_path, 'validation.json')) as json_data:
    validation= json.load(json_data)

validation_img_url = validation['images']
validation_img_url = pd.DataFrame(validation_img_url)
validation_ann = validation['annotations']
validation_ann = pd.DataFrame(validation_ann)
validation = pd.merge(validation_img_url, validation_ann, on='imageId', how='inner')
validation['imageId'] = validation['imageId'].astype(np.uint32)

y_validation = np.array(validation.labelId)
y_validation_bin = binarizer.transform(y_validation)



In [296]:
# Load the generator
session = tf.Session()
training_gen = BatchGenerator(input_dir=images_path_train,
                                  y=y_train_bin[:1000],
                                  epochs=3,
                                  batch_size=64,
                                  session=session)


In [297]:
# Use binary loss instead of categorical loss to penalize each output independently
model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
for i in range(3):
    for batch_x, batch_y in training_gen:
        model.fit(batch_x, batch_y)
        #model.fit(images_path_train, y_train_bin[:1000], epochs=3, batch_size=64)

In [ ]:
model.save('./ensemble_model_1000.h5')